In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp_data_analysis.tables import tbls
import calitp_data_analysis.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
from rt_analysis import rt_filter_map_plot
# import build_speedmaps_index

from IPython.display import display, Markdown, Latex, HTML
import json

In [2]:
## parameters cell
itp_id = 4

In [3]:
# analysis_date = build_speedmaps_index.ANALYSIS_DATE
analysis_date = dt.date(2023, 4, 12)

In [4]:
%%capture_parameters
human_date = analysis_date.strftime('%B %d %Y (%A)')
human_date

{"human_date": "April 12 2023 (Wednesday)"}


In [5]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

AttributeError: reports_site_assessed

In [6]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

NameError: name 'rt_day' is not defined

KeyError: 'organization_name'

# {organization_name}

## About These Maps:

* Each map shows bus (and rail, if applicable) speeds for {organization_name}, with a map each for the morning peak, midday, and afternoon peak periods on {human_date}.
* On the map, routes are split into segments corresponding to the distance between two stops, allowing you to focus on specific portions of the route experiencing a slowdown.
* Route segments are arrow-shaped to indicate direction of travel. Thicker segments indicate more frequent transit routes.
* Hover over, or click, a segment with your mouse to see the exact speed, route name, and transit service frequency in that segment. 
    * Higher-frequency routes (3+ trips per hour) are especially important, since those slowdowns correspond to more vehicles, and riders, delayed in traffic.

## Morning Peak

In [ ]:
%%capture
rt_day.set_filter(start_time='06:00', end_time='09:00')

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

In [ ]:
m

In [ ]:
rt_day.describe_slow_routes()

## Midday

In [ ]:
%%capture
rt_day.set_filter(start_time='10:00', end_time='14:00')

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
if itp_id != 182: # temporarily reduce LA Metro page size
    m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

In [ ]:
m

In [ ]:
rt_day.describe_slow_routes()

## Afternoon Peak

In [ ]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

In [ ]:
m

In [ ]:
rt_day.describe_slow_routes()

In [ ]:
try:
    metrics_df = (rt_day.rt_trips
         >> group_by(_.route_short_name, _.direction_id)
         >> summarize(speed_variance = _.mean_speed_mph.var(),
                     mean_speed_mph = _.mean_speed_mph.mean(),
                     min_hour = _.median_time.min().hour,
                     max_hour = _.median_time.max().hour,
                     n = _.route_short_name.size)
         >> filter(_.min_hour <= 7, _.max_hour >= 15,
                   _.mean_speed_mph < _.mean_speed_mph.quantile(.6),
                   _.speed_variance, _.n > _.n.quantile(.4))
         >> arrange(-_.speed_variance)
         >> head(20)
        )
except Exception as e:
    pass
    # print(e)
## check that route has all-day span...

In [ ]:
def map_from_metrics(rt_day, metrics_df, which_map = 'speeds'):
    def show_title_and_map(which_map):
        try:
            am_map = rt_day.segment_speed_map(how='low_speeds', no_title = True)
            if which_map == 'speeds':
                display(Markdown(f'20th Percentile Vehicle Speeds Between Stops{rt_day.filter_formatted}'))
                display(am_map)
            elif which_map == 'variance':
                display(Markdown(f'Speed Variance Between Stops{rt_day.filter_formatted}'))
                display(rt_day.map_variance(no_title = True))
        except Exception as e:
            display(Markdown(f'Map not available for this time/route'))
            pass
        return
    try:
        route_name = metrics_df.iloc[1,0]
    except Exception as e:
        # print(e)
        return
    rt_day.set_filter(start_time='06:00', end_time='09:00',
                      route_names=[route_name])
    show_title_and_map(which_map)
    
    rt_day.set_filter(start_time='10:00', end_time='14:00',
                      route_names=[route_name])
    show_title_and_map(which_map)

    rt_day.set_filter(start_time='15:00', end_time='19:00',
                      route_names=[route_name])
    show_title_and_map(which_map)

    return

## Daily Speed Patterns

* This chart shows the distribution of median trip speeds throughout the day. It can help find times of day where transit riders experience the slowest overall speeds.

In [ ]:
try:
    rt_day.set_filter(start_time='06:00', end_time='22:00')
    display(Markdown(f'Median Trip Speed by Hour of Day{rt_day.filter_formatted}'))

    rt_day.chart_speeds(no_title=True);
except:
    display(Markdown(f'Chart not available for this agency'))
    pass

## Single-Route Detail Example (Speeds)

* These maps highlight a single route which runs throughout the day, with a low average trip speed and a high amount of variation in speeds between different trips.

In [ ]:
try:
    map_from_metrics(rt_day, metrics_df)
except Exception as e:
    # print(e)
    display(Markdown(f'Example not available for this agency'))
    pass

## Single-Route Detail Example (Variance)

* These maps highlight the same route as above, but showing _variance_ in speeds for each segment instead of 20th percentile speeds.
* Darker segments have higher variance, these are the segments where speeds are most inconsistent between individual trips.
* Segments with high variance (some fast trips and some slow trips) might be better-suited to a different set of projects than segments that experience consistently slow speeds.

In [ ]:
try:
    map_from_metrics(rt_day, metrics_df, which_map = 'variance')
except Exception as e:
    # print(e)
    display(Markdown(f'Example not available for this agency'))
    pass